Here, we do causal inference on pre-compute data.


to get raw prediction please see: /huggingface-model-predict-mnli-tutorial.ipynb

In [9]:
import sys
  
# setting path
sys.path.append('../counterfactual')

import glob
import os
import fuse, causal_utils
from kl_general import sharpness_correction, torch_mult_fuse


TE_CONFIG = {
    "N_LABELS": 3,
    "FUSE": torch_mult_fuse,

    "EPOCHS": 16,
    "BATCH_SIZE": 64,
    "LEARNING_RATE": 0.0001
}


In [10]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from scipy.special import expit

# select data path
data_path='~/debias_nlu/data/nli/'

# select fusion method here
fusion = fuse.sum_fuse

# sharpness correction

In [32]:
df_bias_dev = pd.read_json(
    data_path+'dev_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
bias_dev_score = [b for b in df_bias_dev['bias_probs']]
bias_dev_score = np.array(bias_dev_score)
# y1m0_dev = fusion(avg, bias_dev_score)
df_bert_dev = pd.read_json('~/debias_nlu/data/nli/huggingface/huggingface_roberta_large_mnli_predict_dev_matched.jsonl', lines=True)
y1m1prob_dev = []
for p, h in zip(df_bert_dev['probs'], bias_dev_score):
    new_y1m1 = fusion(np.array(p), h)
    y1m1prob_dev.append(new_y1m1)
c = sharpness_correction(bias_dev_score, y1m1prob_dev) 
c = c*np.ones(3)

In [33]:
c

array([0.11374858, 0.11374858, 0.11374858])

In [34]:
df_bert_dev = pd.read_json('~/debias_nlu/data/nli/huggingface/huggingface_roberta_large_mnli_predict_dev_matched.jsonl', lines=True)
c2 = sharpness_correction(bias_dev_score, df_bert_dev['probs'],config=TE_CONFIG) 
c2 = c2*np.ones(3)

In [35]:
c2

array([0.54552078, 0.54552078, 0.54552078])

#  get score from the bias model

In [36]:
# avg prob
#s score from bias model
df_bias = pd.read_json(data_path+'hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
# df_bias = pd.read_json(data_path+'test_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
hans_score=[b for b in df_bias['bias_probs'] ]
hans_score=np.array(hans_score)

# load main model

In [37]:
# bert model predictions on HANS
df_bert = pd.read_json('~/debias_nlu/data/nli/huggingface/huggingface_roberta_large_mnli_predict_hans.jsonl', lines=True)

In [38]:
# ent = []
y1m1prob = []
for p,h in zip(df_bert['probs'],hans_score):
    new_y1m1 = fusion(np.array(p),h)
    y1m1prob.append(new_y1m1)
  

# corrected y1m0

In [39]:
y1m0prob = fusion(c,hans_score)

# TIE

In [40]:
TIE_A = []
TE_model = []
for p,b in zip(y1m1prob,y1m0prob):
    TIE_A.append(p-b) # TIE  
    
for p,b in zip(df_bert['probs'],hans_score):    
    TE_model.append(p-c2*b)

# Eval HANS

In [41]:
key = {0:"entailment",1:"contradiction",2:"neutral"}
t_labels = []
for i in df_bert['probs']:
    t_labels.append(key[np.argmax(i)])
df_bert['label']=t_labels

labels = []
for i in TIE_A:
    labels.append(key[np.argmax(i)])
df_bert['TIE_debias_label']=labels


labels = []
for i in TE_model:
    labels.append(key[np.argmax(i)])
df_bert['TE_debias_label']=labels


In [42]:
text_ans=""
for idx, obj in enumerate(df_bert['label']):
    text_ans = text_ans + "ex"+str(idx)+","+obj+"\n"   
    
text_ans_debias_tie=""
for idx, obj in enumerate(df_bert['TIE_debias_label']):
    text_ans_debias_tie = text_ans_debias_tie + "ex"+str(idx)+","+obj+"\n"          
    
text_ans_debias_te=""
for idx, obj in enumerate(df_bert['TE_debias_label']):
    text_ans_debias_te = text_ans_debias_te + "ex"+str(idx)+","+obj+"\n"           

In [43]:
def format_label(label):
    if label == "entailment":
        return "entailment"
    else:
        return "non-entailment"

guess_dict = {}
for line in text_ans.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict[parts[0]] = format_label(parts[1])
        
guess_dict_debias_tie = {}
for line in text_ans_debias_tie.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict_debias_tie[parts[0]] = format_label(parts[1])        
        
 
guess_dict_debias_te = {}
for line in text_ans_debias_te.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict_debias_te[parts[0]] = format_label(parts[1])     

In [44]:
labels,baseline_avg=causal_utils.get_heur(guess_dict) # normal

Heuristic entailed results:
lexical_overlap: 0.9998
subsequence: 1.0
constituent: 0.9898

Heuristic non-entailed results:
lexical_overlap: 0.8938
subsequence: 0.3378
constituent: 0.1668
avg: 0.7313333333333335


In [45]:
labels,debias_avg=causal_utils.get_heur(guess_dict_debias_tie) # TIE_A

Heuristic entailed results:
lexical_overlap: 0.9994
subsequence: 1.0
constituent: 0.9886

Heuristic non-entailed results:
lexical_overlap: 0.9066
subsequence: 0.3562
constituent: 0.1866
avg: 0.7395666666666667


In [46]:
labels,debias_avg=causal_utils.get_heur(guess_dict_debias_te) # TE_model

Heuristic entailed results:
lexical_overlap: 0.9994
subsequence: 1.0
constituent: 0.9858

Heuristic non-entailed results:
lexical_overlap: 0.924
subsequence: 0.3788
constituent: 0.2164
avg: 0.7507333333333334
